# Linear Programming

## Task

Maximize:

$ z = x_1 + 10x_2 $

Constraints:

- $ x_1 \geq 0 $
- $ x_2 \geq 0 $
- $ 0.2x_1 + 4 \geq x_2 $
- $ -0.2x_1 + 6 \geq x_2 $
- $ 10x_1 + 1 \geq x_2 $

## Code

### Import Package

In [1]:
import pyomo.environ as pyomo

### Creating a Pyomo Model

In Pyomo, a model is a container that holds:
- variables
- parameters (data)
- constraints
- objectives

We start by creating a **ConcreteModel**, which means:
- the data is defined immediately
- the model is built as the code runs
- this is the most common and beginner-friendly approach

Most Pyomo examples and learning materials use `ConcreteModel`.

#### Why ConcreteModel?

Pyomo has two main model types:

- **ConcreteModel**
  - Data is defined immediately
  - Easy to understand
  - Best for learning and experimentation

- **AbstractModel**
  - Model structure is defined first
  - Data is loaded later from files
  - Used mainly for very large or production models

In this notebook, we use `ConcreteModel` because it is simpler and clearer.

In [2]:
model = pyomo.ConcreteModel()

<a id='Creating_Variables'></a>
### Creating Variables (Decision Variables)

A **decision variable** is a value that the solver is allowed to choose.

In this model:

* **x1** → a value the solver can change
* **x2** → another value the solver can change

In Pyomo, decision variables are created as **attributes of the model** using `pyomo.Var()`.

<br>
Each variable must also have a **domain**, which defines the type of values it can take (for example, non-negative, integer, binary, etc.).

```python
model.x1 = pyomo.Var(domain=pyomo.NonNegativeReals)
model.x2 = pyomo.Var(domain=pyomo.NonNegativeReals)
```

Here, `pyomo.NonNegativeReals` means the variable:

* is a real number
* must satisfy ($ x \ge 0 $)

So this line tells the model:

> “Create a decision variable called `x1` whose value must be a non-negative real number.”

The solver’s job is to choose values for these decision variables that:

1. satisfy all constraints, and
2. optimize the objective function.

In [3]:
# In Pyomo, everything lives inside the model as attributes.
# model.attribute = pyomo.Var(domain=pyomo.<DOMAIN>)
model.x1 = pyomo.Var(domain = pyomo.NonNegativeReals) # this makex x1 >= 0
model.x2 = pyomo.Var(domain = pyomo.NonNegativeReals) # this makex x2 >= 0

### Creating Constraints

Constraints define the rules that decision variables must follow.

In Pyomo, constraints are **symbolic mathematical expressions** that must be satisfied by
any solution chosen by the solver.

---

#### 1. ConstraintList (add constraints one by one)

We use `ConstraintList()` when we want to add constraints dynamically or one at a time.

```python
model.c = pyomo.ConstraintList()
model.c.add(0.2 * model.x1 + 4 >= model.x2)
model.c.add(-0.2 * model.x1 + 6 >= model.x2)
model.c.add(10 * model.x1 + 1 >= model.x2)
```

This is useful for:

* Prototyping
* Small models
* When the number of constraints is not fixed

---

#### 2. Named / rule-based constraints

We can define a constraint using a **rule (function)** and assign it to the model.

```python
def constraint_rule(model):
    return model.x2 <= 0.2 * model.x1 + 4

model.c1 = pyomo.Constraint(rule=constraint_rule)
```

<b style="color:red">NOTE</b>: The function **must return a mathematical expression**, not `True` or `False`.

---

#### 3. Indexed constraints (pattern-based, scalable)

If constraints follow a pattern, we can use **indexed constraints**, which scale well to
large models and closely match mathematical notation.

```python
model.I = pyomo.RangeSet(3)

def rule(model, i):
    if i == 1:
        return model.x2 <= 0.2 * model.x1 + 4
    elif i == 2:
        return model.x2 <= -0.2 * model.x1 + 6
    else:
        return model.x2 <= 10 * model.x1 + 1

model.c = pyomo.Constraint(model.I, rule=rule)
```



In [4]:
# In this example we would use constraintlist

model.c = pyomo.ConstraintList()
model.c.add(0.2 * model.x1 + 4 >= model.x2)
model.c.add(-0.2 * model.x1 + 6 >= model.x2)
model.c.add(10 * model.x1 + 1 >= model.x2)

### Creating the Objective Function
The objective function defines what the model is trying to optimize.<br>
In Pyomo, we create an objective using pyomo.Objective().

It requires two main components:
- rule → the mathematical expression to optimize
- sense → whether to maximize or minimize the expression

<b style="color:green">Key Notes</b>:
- The objective function is symbolic, just like constraints
- Pyomo does not compute the value immediately
- The solver decides the variable values later

In [5]:
# we will be using the pyomo.Objective() function where it accepts the rule (the objective function) and the sense (do we want to maximize or minimize the function)

model.objective = pyomo.Objective(rule = lambda model: model.x1 + 10 * model.x2, 
                                  sense = pyomo.maximize)


This tells the solver:

“Pick values for x1 and x2 that maximize
$ x_1 + 10x_2 $
subject to all constraints.”

#### Other Ways to Create the Objective
besides using the lambda function in making the objective function there are other ways to create the objecive function

##### 1. Defining Rule Function
```python
def obj_rule(model):
    return model.x1 + 10 * model.x2

model.objective = pyomo.Objective(rule=obj_rule, 
                                  sense=pyomo.maximize
                                  )
```
---

##### 2. Directly Making the Objective Function
```python
model.objective = pyomo.Objective(expr=model.x1 + 10 * model.x2,
                                  sense=pyomo.maximize
                                  )
```
| Keyword | What it expects    | When to use                                 |
| ------- | ------------------ | ------------------------------------------- |
| `rule=` | A function         | When logic depends on indices or conditions |
| `expr=` | A Pyomo expression | When objective is simple and fixed          |


### Creating the Solver

Pyomo:

* **does NOT solve** optimization problems itself
* it **builds the mathematical model**
* then **hands it to a solver**

---

#### Types of Solver
| Term  | Complete                                 |
| ----- | ---------------------------------------- |
| LP    | Linear Programming                       |
| MILP  | **Mixed-Integer** Linear Programming     |
| QP    | Quadratic Programming                    |
| NLP   | **Non-Linear** Programming               |
| MINLP | **Mixed-Integer Non-Linear** Programming |


##### Open-source solvers
| Solver | Task it Solves |
| ------ | -------------- |
| **GLPK** | LP, MILP |
| **CBC** | MILP |
| **HiGHS** | LP, MILP, QP |
| **Ipopt** | NLP (continuous only) |
| **Couenne** | MINLP |
| **SCIP** | MILP, MINLP |
| **APOPT** | NLP, MINLP |

##### Commercial solvers (licensed)
| Solver | Task it Solves |
| ------ | -------------- |
| **Gurobi** | LP, QP, MILP |
| **CPLEX** | LP, QP, MILP |
| **KNITRO** | NLP, MINLP |
| **BARON** | global NLP, MINLP |
| **MOSEK** | LP, QP, conic | 

---

#### <b style="color:red">IMPORTANT NOTE</b> something beginners often miss

The solver **must already be installed** on your system.

Example checks:
```bash
glpsol --version
ipopt -v
```

If Pyomo can’t find the solver, it will error.

some common solver documentation links:<br>
[![GLPK](https://img.shields.io/badge/GLPK-blue?style=plastic)](https://www.gnu.org/software/glpk/#downloading)<br>
[![IPOPT](https://img.shields.io/badge/IPOPT-blue?style=plastic)](https://coin-or.github.io/Ipopt/INSTALL.html)

In [6]:
solver = pyomo.SolverFactory('glpk')

This means:

> “Hey Pyomo, I want to use **GLPK** to solve this model.”

Nothing is solved yet, we’re just **choosing the solver**.

### Getting the Results

In [14]:
from pyomo.environ import value
result = solver.solve(model)

print(result)
print(f"x1 = {value(model.x1)}")
print(f"x2 = {value(model.x2)}")
print()

#you can check the solver status by doing
print(f"solver status: {result.solver.status}")


Problem: 
- Name: unknown
  Lower bound: 55.0
  Upper bound: 55.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.002721548080444336
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

x1 = 5.00000000000002
x2 = 5.0

solver status: ok


In Pyomo, variables are symbolic objects.<br>
After solving, their optimal numerical values are stored inside them.<br>
To retrieve the actual number, we must explicitly extract the value.<br>